In [1]:
import transformers
import torch 
import gc 
import json 
from tokens import * 

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [01:14<00:00, 24.95s/it]
/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-11-16 16:35:57.959161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical

In [4]:
def repeater(messages):

    # Apply chat template to messages
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Define terminators
    terminators = [
        pipeline.tokenizer.eos_token_id,
    ]

    # Generate text
    outputs = pipeline(
        prompt,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        max_new_tokens=2024*2
    )
    return len(prompt), outputs

In [5]:
with open('articles.json', 'r') as file: 
    articles = json.load(file)

print(f'{len(articles[len(articles) - 1])} are available.')

5 are available.


In [6]:
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing NEUTRAL news article based on the provided article information."
                                  "Try to filter the bias that maybe existent in the article" 
                                  "Write a new article as a journalist, ensuring the content is unbiased, factual, and informative."},
                                
    {"role": "user", "content": articles[0][0]}
]

In [7]:
prompt_len, outputs = repeater(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [9]:
print(outputs[0]["generated_text"][prompt_len:], end='.')

with open('mistral.txt', 'w') as file: 
    file.write(outputs[0]["generated_text"][prompt_len:])

 Title: Keir Starmer Revises Ministerial Gift and Hospitality Rules in Response to Public Scrutiny

Keir Starmer, the Prime Minister, has amended the rules governing gifts and hospitality received by ministers, following a series of criticisms over freebies accepted by senior Labour figures. The updated ministerial code emphasizes the importance of maintaining public trust in the propriety of ministers' actions.

Under the revised code, ministers are expected to consider the need to uphold public confidence when deciding whether to accept gifts or hospitality. A monthly register of gifts received by ministers, valued at over £140, will be published, as opposed to the previous quarterly schedule.

The changes come after a backlash over gifts received by Starmer, including tickets to see Taylor Swift, and clothing from Labour peer and donor Lord Alli. In response, Starmer, Deputy Prime Minister Angela Rayner, and Chancellor Rachel Reeves have ceased accepting donated clothing.

The updat

In [13]:
del outputs
gc.collect()
torch.cuda.empty_cache()

In [21]:
del model